In [16]:
from arcgis.gis import GIS

In [17]:
gis = GIS(profile='sgc20')

In [ ]:
#parcel_service = "6263a83b09fd42cc94accf210dc4f646"
#search_result = gis.content.get(parcel_service)
import sgccat20lib
PARCEL_SERVICE_NAME = "PARCEL_SERVICE_NAME"
dictParams = sgccat20lib.ObtenerParametros()
parcel_service = dictParams[PARCEL_SERVICE_NAME]
search_result = gis.content.get(parcel_service)
numero_tramite = "08001-2021-MutM-M"
versionTramite = "adminsgcv2.CON_{}".format(numero_tramite)
prefixlog = 'concil_{}.txt'.format(numero_tramite)
dirLogs = r"C:\Logs\mutacion\uno"
logfile = os.path.join(dirLogs,prefixlog)

In [ ]:
sgccat20lib.reconciliar_versionTramite_arcpy(parcel_service, versionTramite, logfile)

In [ ]:
from arcgis.features import FeatureLayerCollection
p_flc = FeatureLayerCollection.fromitem(search_result)
version_manager = p_flc.versions

In [ ]:
#locks = version_manager.locks
#locks
bloqueos = version_manager.locks if version_manager.locks else None
bloqueos
#if bloqueos:
for version_locked in version_manager.locks:
    # print(version_locked.properties.versionName)
    if version_locked.properties.versionName == versionTramite:
        print("version bloqueada")
        desbloqueada = version_manager.purge(version=versionTramite)
        if desbloqueada is False:
            raise Exception("No se pudo desbloquear la versión!.")
        print("desbloqueada: "+str(desbloqueada))

In [ ]:
desbloqueada = True
postear = True
if desbloqueada:
    version = version_manager.get(version=versionTramite, mode="edit")
    resultado = version.reconcile(with_post=postear)
    print("Resultado reconciliación y posteo:{}".format("resultado"))
    if resultado:
        version_borrada = version.delete()
        if version_borrada is False:
            print("No fue posible borrar la versión")

In [ ]:
def conciliarVersion(postear):
    try:
        if bool(postear) is True:
            sgccat20lib.reconciliar_versionTramite(parcel_service, versionTramite)
        else:
            sgccat20lib.reconciliar_versionTramite(parcel_service, versionTramite, False)
        print("Conciliaciom terminada correctamente.")
    except Exception as e:
        print("Error: {}".format(e))

In [ ]:
print(versionTramite)
conciliarVersion(True)

In [ ]:
sgccat20lib.eliminarVersion(parcel_service, versionTramite)

In [15]:
import os
logfile = r"C:\Logs\mutacion\uno\concil_wrmr.txt"
pathLog = os.path.dirname(logfile) ## directory of file
print(pathLog)
if not os.path.exists(pathLog):
    os.makedirs(pathLog)
if os.path.exists(logfile):
    os.remove(logfile)

C:\Logs\mutacion\uno


In [20]:
from arcgis.geoprocessing import import_toolbox
urlToolBox = gis.content.search(query="title:{}".format("inforesolucion"), item_type="Geoprocessing Toolbox")
thetool = import_toolbox(url_or_item=urlToolBox[0], gis=gis)
len(urlToolBox)

1